In [1]:
import json
import math
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely import geometry
from shapely.geometry import Polygon, LineString, Point, shape
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
from numpy import nan
import statistics

In [32]:
#set the file path 
path='/Users/paw_paw/Library/Mobile Documents/com~apple~CloudDocs/学习/PC5287_M.SC Coursework Thesis for Physics And Technology/data set'

# read the raw GPR data into a pandas dataframe 
data = pd.read_json(path+'/SG_landuse_wGPR.json') # data is the GPR data 

check the datatype of the 2 columns...but it turns out that they are all list.

In [3]:
def extract_element(desc_pa,element):
    return desc_pa.get(element)

extract all features 

In [33]:
for key in data['attributes'][0].keys():
    data[key]=data.attributes.apply(extract_element,args=(key,))

In [34]:
data

,attributes,geometry,OBJECTID,LU_DESC,LU_TEXT,LU_DT_DESC,GPR,GPR_NUM,WHI_Q_MX,GPR_B_MN,...,PLN_AREA_N,SUBZONE_NO,SUBZONE_N,CREATE_BY,CREATE_DT,LST_MDF_BY,LST_MDF_DT,SYSTEM_ID,SHAPE_Length,SHAPE_Area
0,"{'OBJECTID': 108481, 'LU_DESC': 'BUSINESS 2', ...","{'rings': [[[103.6141365232126, 1.252949266554...",108481,BUSINESS 2,None,None,1.0,1.0,NaN,NaN,...,TUAS,6,TUAS VIEW EXTENSION,Poh Sock Geck,1548851081000,Poh Sock Geck,1548851081000,300119-176077734734,5204.536600,735026.472606
1,"{'OBJECTID': 108482, 'LU_DESC': 'BUSINESS 2', ...","{'rings': [[[103.61413257364292, 1.26050061227...",108482,BUSINESS 2,None,None,1.0,1.0,NaN,NaN,...,TUAS,6,TUAS VIEW EXTENSION,Poh Sock Geck,1548851081000,Poh Sock Geck,1548851081000,300119-548854630600,3543.900652,345914.636037
2,"{'OBJECTID': 108669, 'LU_DESC': 'BUSINESS 2', ...","{'rings': [[[103.61413257364292, 1.26050061227...",108669,BUSINESS 2,None,None,1.0,1.0,NaN,NaN,...,TUAS,6,TUAS VIEW EXTENSION,Poh Sock Geck,1548851087000,Poh Sock Geck,1548851087000,300119-648882633981,1895.091515,9411.081297
3,"{'OBJECTID': 108617, 'LU_DESC': 'BUSINESS 2', ...","{'rings': [[[103.61084243369223, 1.27424179894...",108617,BUSINESS 2,None,None,1.0,1.0,NaN,NaN,...,TUAS,6,TUAS VIEW EXTENSION,Poh Sock Geck,1548851085000,Poh Sock Geck,1548851085000,300119-855138884619,5368.220769,966310.427138
4,"{'OBJECTID': 108505, 'LU_DESC': 'BUSINESS 2', ...","{'rings': [[[103.61637440251737, 1.27216319344...",108505,BUSINESS 2,None,None,1.0,1.0,NaN,NaN,...,TUAS,6,TUAS VIEW EXTENSION,Poh Sock Geck,1548851081000,Poh Sock Geck,1548851081000,300119-308340339409,1458.621768,8077.741680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33112,"{'OBJECTID': 110388, 'LU_DESC': 'RESIDENTIAL',...","{'rings': [[[103.9048849196422, 1.311382377246...",110388,RESIDENTIAL,,PRIVATE HOUSING - NON-LANDED,1.4,1.4,NaN,NaN,...,MARINE PARADE,2,KATONG,"LIM PEI JUAN, TRACY",1187083401000,Kamsani Bin Afandi,1571136105000,180718-11925,68.643626,288.225272
33113,"{'OBJECTID': 110401, 'LU_DESC': 'RESIDENTIAL',...","{'rings': [[[103.90458099712198, 1.31143812192...",110401,RESIDENTIAL,,PRIVATE HOUSING - NON-LANDED,1.4,1.4,NaN,NaN,...,MARINE PARADE,2,KATONG,"LIM PEI JUAN, TRACY",1187083401000,Kamsani Bin Afandi,1571136105000,180718-11926,52.901060,130.410105
33114,"{'OBJECTID': 111503, 'LU_DESC': 'RESIDENTIAL',...","{'rings': [[[103.90471532402312, 1.31125518331...",111503,RESIDENTIAL,,PRIVATE HOUSING - NON-LANDED,1.4,1.4,NaN,NaN,...,MARINE PARADE,2,KATONG,"LIM PEI JUAN, TRACY",1187083693000,Kamsani Bin Afandi,1571136111000,180718-12230,63.457515,164.929047
33115,"{'OBJECTID': 111504, 'LU_DESC': 'RESIDENTIAL',...","{'rings': [[[103.90466856255233, 1.31121926332...",111504,RESIDENTIAL,,PRIVATE HOUSING - NON-LANDED,1.4,1.4,NaN,NaN,...,MARINE PARADE,2,KATONG,"LIM PEI JUAN, TRACY",1187084260000,Kamsani Bin Afandi,1571136111000,180718-13187,61.342395,158.483763


Dont know why the comma needs to be added     
See the artical on Zhihu: [pandas apply 传入参数](https://zhuanlan.zhihu.com/p/55073568)

In [38]:
pd.DataFrame(data.dtypes,columns=['type'])

,type
attributes,object
geometry,object
OBJECTID,int64
LU_DESC,object
LU_TEXT,object
LU_DT_DESC,object
GPR,object
GPR_NUM,float64
WHI_Q_MX,float64
GPR_B_MN,float64


In [49]:
# GPR number * shape of the area = the designed population density 
data['POP_DENS'] = data.apply(lambda x: x['GPR_NUM'] * x['SHAPE_Area'], axis=1)

In [50]:
# creat a new column to store the sorted out geometry data (boaders of the GPR areas), 
# as the currently on cannot straghtly be turned into geomenty feature

def get_list(geometry):
    return geometry.get('rings')[0]

data['Geo'] = data['geometry'].apply(get_list)

In [61]:
data.set_geometry(data['geometry'])

TypeError: Input must be valid geometry objects: {'rings': [[['103.61413652321259', '1.25294926655456'], ['103.61413652321650', '1.25294921952792'], ['103.61413633477807', '1.25294619714603'], ['103.61413634920434', '1.25265367516214'], ['103.61414221957594', '1.25240099789647'], ['103.61415976259363', '1.25214886513166'], ['103.61418895844034', '1.25189783485439'], ['103.61422973518665', '1.25164844334225'], ['103.61428200562794', '1.25140123320192'], ['103.61434564931464', '1.25115674522840'], ['103.61442458975021', '1.25087567271147'], ['103.61449499065475', '1.25058330584295'], ['103.61453149163530', '1.25038374772811'], ['103.61457522003388', '1.25001383801084'], ['103.61457606487629', '1.25000016960692'], ['103.61457687598704', '1.24999237862621'], ['103.61457461267818', '1.24998072577687'], ['103.61457993726458', '1.24989445584597'], ['103.61458232064216', '1.24985583812795'], ['103.61458381418853', '1.24983164847124'], ['103.61458690655087', '1.24978154274621'], ['103.614588852

We can see that the precision of the numbers are not all the same. Let's set them all the same. 

In [53]:
def set_precision(geo_list):
    for element in geo_list:
        element[0] = ("%.14f" % float(element[0]))
        element[1] = ("%.14f" % float(element[1]))
    return geo_list

In [54]:
data['Geo']=data['Geo'].apply(set_precision)

In [55]:
data['Geo']

0        [[103.61413652321259, 1.25294926655456], [103....
1        [[103.61413257364292, 1.26050061227085], [103....
2        [[103.61413257364292, 1.26050061227085], [103....
3        [[103.61084243369223, 1.27424179894692], [103....
4        [[103.61637440251737, 1.27216319344277], [103....
                               ...                        
33112    [[103.90488491964220, 1.31138237724605], [103....
33113    [[103.90458099712198, 1.31143812192426], [103....
33114    [[103.90471532402312, 1.31125518331999], [103....
33115    [[103.90466856255233, 1.31121926332259], [103....
33116    [[103.90499449908609, 1.31175005503570], [103....
Name: Geo, Length: 33117, dtype: object

In [56]:
def get_polygon(point_list):
    temp_df = pd.DataFrame(point_list)
    temp_gdf = gpd.GeoDataFrame(temp_df, geometry = gpd.points_from_xy(temp_df[0],temp_df[1]))
    poly = geometry.Polygon([[p.x, p.y] for p in temp_gdf.geometry])
    return poly 

https://stackoverflow.com/questions/30457089/how-to-create-a-shapely-polygon-from-a-list-of-shapely-points

In [57]:
data['ply'] = data.Geo.apply(get_polygon)

In [ ]:
data

,attributes,geometry,GPR,GPR_NUM,PLN_AREA_N,SUBZONE_N,SHAPE_Area,LU_DESC,POP_DENS,Geo,ply
0,"{'OBJECTID': 108481, 'LU_DESC': 'BUSINESS 2', ...","{'rings': [[['103.61413652321259', '1.25294926...",1.0,1.0,TUAS,TUAS VIEW EXTENSION,735026.472606,BUSINESS 2,735026.472606,"[[103.61413652321259, 1.25294926655456], [103....","POLYGON ((103.6141365232126 1.25294926655456, ..."
1,"{'OBJECTID': 108482, 'LU_DESC': 'BUSINESS 2', ...","{'rings': [[['103.61413257364292', '1.26050061...",1.0,1.0,TUAS,TUAS VIEW EXTENSION,345914.636037,BUSINESS 2,345914.636037,"[[103.61413257364292, 1.26050061227085], [103....","POLYGON ((103.61413257364292 1.26050061227085,..."
2,"{'OBJECTID': 108669, 'LU_DESC': 'BUSINESS 2', ...","{'rings': [[['103.61413257364292', '1.26050061...",1.0,1.0,TUAS,TUAS VIEW EXTENSION,9411.081297,BUSINESS 2,9411.081297,"[[103.61413257364292, 1.26050061227085], [103....","POLYGON ((103.61413257364292 1.26050061227085,..."
3,"{'OBJECTID': 108617, 'LU_DESC': 'BUSINESS 2', ...","{'rings': [[['103.61084243369223', '1.27424179...",1.0,1.0,TUAS,TUAS VIEW EXTENSION,966310.427138,BUSINESS 2,966310.427138,"[[103.61084243369223, 1.27424179894692], [103....","POLYGON ((103.61084243369223 1.27424179894692,..."
4,"{'OBJECTID': 108505, 'LU_DESC': 'BUSINESS 2', ...","{'rings': [[['103.61637440251737', '1.27216319...",1.0,1.0,TUAS,TUAS VIEW EXTENSION,8077.741680,BUSINESS 2,8077.741680,"[[103.61637440251737, 1.27216319344277], [103....","POLYGON ((103.61637440251737 1.27216319344277,..."
...,...,...,...,...,...,...,...,...,...,...,...
33112,"{'OBJECTID': 110388, 'LU_DESC': 'RESIDENTIAL',...","{'rings': [[['103.90488491964220', '1.31138237...",1.4,1.4,MARINE PARADE,KATONG,288.225272,RESIDENTIAL,403.515381,"[[103.90488491964220, 1.31138237724605], [103....","POLYGON ((103.9048849196422 1.31138237724605, ..."
33113,"{'OBJECTID': 110401, 'LU_DESC': 'RESIDENTIAL',...","{'rings': [[['103.90458099712198', '1.31143812...",1.4,1.4,MARINE PARADE,KATONG,130.410105,RESIDENTIAL,182.574148,"[[103.90458099712198, 1.31143812192426], [103....","POLYGON ((103.90458099712198 1.31143812192426,..."
33114,"{'OBJECTID': 111503, 'LU_DESC': 'RESIDENTIAL',...","{'rings': [[['103.90471532402312', '1.31125518...",1.4,1.4,MARINE PARADE,KATONG,164.929047,RESIDENTIAL,230.900666,"[[103.90471532402312, 1.31125518331999], [103....","POLYGON ((103.90471532402312 1.31125518331999,..."
33115,"{'OBJECTID': 111504, 'LU_DESC': 'RESIDENTIAL',...","{'rings': [[['103.90466856255233', '1.31121926...",1.4,1.4,MARINE PARADE,KATONG,158.483763,RESIDENTIAL,221.877268,"[[103.90466856255233, 1.31121926332259], [103....","POLYGON ((103.90466856255233 1.31121926332259,..."


In [63]:
gpr = data.drop(columns=['geometry','Geo'])

In [64]:
gpr = gpr.rename(columns={'ply':'geometry'})

In [65]:
gpr = gpd.GeoDataFrame(gpr)

In [66]:
for i in range(0,33117):
    if pd.isna(gpr.loc[i,'GPR_NUM'])==True:
        gpr.loc[i,'POP_DENS'] = gpr.loc[i,'SHAPE_Area']*float(gpr.loc[i,'GPR'])

for i in range(0,33117):
    if pd.isna(gpr.loc[i,'POP_DENS'])==True:
        print(gpr.iloc[i])

In [67]:
gpr.set_geometry('geometry')

,attributes,OBJECTID,LU_DESC,LU_TEXT,LU_DT_DESC,GPR,GPR_NUM,WHI_Q_MX,GPR_B_MN,REGION_N,...,SUBZONE_N,CREATE_BY,CREATE_DT,LST_MDF_BY,LST_MDF_DT,SYSTEM_ID,SHAPE_Length,SHAPE_Area,POP_DENS,geometry
0,"{'OBJECTID': 108481, 'LU_DESC': 'BUSINESS 2', ...",108481,BUSINESS 2,None,None,1.0,1.0,NaN,NaN,WEST REGION,...,TUAS VIEW EXTENSION,Poh Sock Geck,1548851081000,Poh Sock Geck,1548851081000,300119-176077734734,5204.536600,735026.472606,735026.472606,"POLYGON ((103.61414 1.25295, 103.61414 1.25295..."
1,"{'OBJECTID': 108482, 'LU_DESC': 'BUSINESS 2', ...",108482,BUSINESS 2,None,None,1.0,1.0,NaN,NaN,WEST REGION,...,TUAS VIEW EXTENSION,Poh Sock Geck,1548851081000,Poh Sock Geck,1548851081000,300119-548854630600,3543.900652,345914.636037,345914.636037,"POLYGON ((103.61413 1.26050, 103.61414 1.25295..."
2,"{'OBJECTID': 108669, 'LU_DESC': 'BUSINESS 2', ...",108669,BUSINESS 2,None,None,1.0,1.0,NaN,NaN,WEST REGION,...,TUAS VIEW EXTENSION,Poh Sock Geck,1548851087000,Poh Sock Geck,1548851087000,300119-648882633981,1895.091515,9411.081297,9411.081297,"POLYGON ((103.61413 1.26050, 103.60571 1.26050..."
3,"{'OBJECTID': 108617, 'LU_DESC': 'BUSINESS 2', ...",108617,BUSINESS 2,None,None,1.0,1.0,NaN,NaN,WEST REGION,...,TUAS VIEW EXTENSION,Poh Sock Geck,1548851085000,Poh Sock Geck,1548851085000,300119-855138884619,5368.220769,966310.427138,966310.427138,"POLYGON ((103.61084 1.27424, 103.61638 1.27216..."
4,"{'OBJECTID': 108505, 'LU_DESC': 'BUSINESS 2', ...",108505,BUSINESS 2,None,None,1.0,1.0,NaN,NaN,WEST REGION,...,TUAS VIEW EXTENSION,Poh Sock Geck,1548851081000,Poh Sock Geck,1548851081000,300119-308340339409,1458.621768,8077.741680,8077.741680,"POLYGON ((103.61637 1.27216, 103.61638 1.27216..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33112,"{'OBJECTID': 110388, 'LU_DESC': 'RESIDENTIAL',...",110388,RESIDENTIAL,,PRIVATE HOUSING - NON-LANDED,1.4,1.4,NaN,NaN,CENTRAL REGION,...,KATONG,"LIM PEI JUAN, TRACY",1187083401000,Kamsani Bin Afandi,1571136105000,180718-11925,68.643626,288.225272,403.515381,"POLYGON ((103.90488 1.31138, 103.90478 1.31130..."
33113,"{'OBJECTID': 110401, 'LU_DESC': 'RESIDENTIAL',...",110401,RESIDENTIAL,,PRIVATE HOUSING - NON-LANDED,1.4,1.4,NaN,NaN,CENTRAL REGION,...,KATONG,"LIM PEI JUAN, TRACY",1187083401000,Kamsani Bin Afandi,1571136105000,180718-11926,52.901060,130.410105,182.574148,"POLYGON ((103.90458 1.31144, 103.90453 1.31140..."
33114,"{'OBJECTID': 111503, 'LU_DESC': 'RESIDENTIAL',...",111503,RESIDENTIAL,,PRIVATE HOUSING - NON-LANDED,1.4,1.4,NaN,NaN,CENTRAL REGION,...,KATONG,"LIM PEI JUAN, TRACY",1187083693000,Kamsani Bin Afandi,1571136111000,180718-12230,63.457515,164.929047,230.900666,"POLYGON ((103.90472 1.31126, 103.90467 1.31122..."
33115,"{'OBJECTID': 111504, 'LU_DESC': 'RESIDENTIAL',...",111504,RESIDENTIAL,,PRIVATE HOUSING - NON-LANDED,1.4,1.4,NaN,NaN,CENTRAL REGION,...,KATONG,"LIM PEI JUAN, TRACY",1187084260000,Kamsani Bin Afandi,1571136111000,180718-13187,61.342395,158.483763,221.877268,"POLYGON ((103.90467 1.31122, 103.90465 1.31120..."


In [68]:
gpr = gpr.set_crs(epsg=4326,allow_override=False)
gpr = gpr.to_crs(3414)

In [69]:
gpr

,attributes,OBJECTID,LU_DESC,LU_TEXT,LU_DT_DESC,GPR,GPR_NUM,WHI_Q_MX,GPR_B_MN,REGION_N,...,SUBZONE_N,CREATE_BY,CREATE_DT,LST_MDF_BY,LST_MDF_DT,SYSTEM_ID,SHAPE_Length,SHAPE_Area,POP_DENS,geometry
0,"{'OBJECTID': 108481, 'LU_DESC': 'BUSINESS 2', ...",108481,BUSINESS 2,None,None,1.0,1.0,NaN,NaN,WEST REGION,...,TUAS VIEW EXTENSION,Poh Sock Geck,1548851081000,Poh Sock Geck,1548851081000,300119-176077734734,5204.536600,735026.472606,735026.472606,"POLYGON ((3606.500 26171.307, 3606.500 26171.3..."
1,"{'OBJECTID': 108482, 'LU_DESC': 'BUSINESS 2', ...",108482,BUSINESS 2,None,None,1.0,1.0,NaN,NaN,WEST REGION,...,TUAS VIEW EXTENSION,Poh Sock Geck,1548851081000,Poh Sock Geck,1548851081000,300119-548854630600,3543.900652,345914.636037,345914.636037,"POLYGON ((3606.131 27006.302, 3606.500 26171.3..."
2,"{'OBJECTID': 108669, 'LU_DESC': 'BUSINESS 2', ...",108669,BUSINESS 2,None,None,1.0,1.0,NaN,NaN,WEST REGION,...,TUAS VIEW EXTENSION,Poh Sock Geck,1548851087000,Poh Sock Geck,1548851087000,300119-648882633981,1895.091515,9411.081297,9411.081297,"POLYGON ((3606.131 27006.302, 2668.620 27006.3..."
3,"{'OBJECTID': 108617, 'LU_DESC': 'BUSINESS 2', ...",108617,BUSINESS 2,None,None,1.0,1.0,NaN,NaN,WEST REGION,...,TUAS VIEW EXTENSION,Poh Sock Geck,1548851085000,Poh Sock Geck,1548851085000,300119-855138884619,5368.220769,966310.427138,966310.427138,"POLYGON ((3240.090 28525.774, 3856.615 28295.2..."
4,"{'OBJECTID': 108505, 'LU_DESC': 'BUSINESS 2', ...",108505,BUSINESS 2,None,None,1.0,1.0,NaN,NaN,WEST REGION,...,TUAS VIEW EXTENSION,Poh Sock Geck,1548851081000,Poh Sock Geck,1548851081000,300119-308340339409,1458.621768,8077.741680,8077.741680,"POLYGON ((3855.740 28295.878, 3856.615 28295.2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33112,"{'OBJECTID': 110388, 'LU_DESC': 'RESIDENTIAL',...",110388,RESIDENTIAL,,PRIVATE HOUSING - NON-LANDED,1.4,1.4,NaN,NaN,CENTRAL REGION,...,KATONG,"LIM PEI JUAN, TRACY",1187083401000,Kamsani Bin Afandi,1571136105000,180718-11925,68.643626,288.225272,403.515381,"POLYGON ((35964.658 32631.632, 35952.880 32622..."
33113,"{'OBJECTID': 110401, 'LU_DESC': 'RESIDENTIAL',...",110401,RESIDENTIAL,,PRIVATE HOUSING - NON-LANDED,1.4,1.4,NaN,NaN,CENTRAL REGION,...,KATONG,"LIM PEI JUAN, TRACY",1187083401000,Kamsani Bin Afandi,1571136105000,180718-11926,52.901060,130.410105,182.574148,"POLYGON ((35930.834 32637.795, 35925.593 32633..."
33114,"{'OBJECTID': 111503, 'LU_DESC': 'RESIDENTIAL',...",111503,RESIDENTIAL,,PRIVATE HOUSING - NON-LANDED,1.4,1.4,NaN,NaN,CENTRAL REGION,...,KATONG,"LIM PEI JUAN, TRACY",1187083693000,Kamsani Bin Afandi,1571136111000,180718-12230,63.457515,164.929047,230.900666,"POLYGON ((35945.784 32617.567, 35940.580 32613..."
33115,"{'OBJECTID': 111504, 'LU_DESC': 'RESIDENTIAL',...",111504,RESIDENTIAL,,PRIVATE HOUSING - NON-LANDED,1.4,1.4,NaN,NaN,CENTRAL REGION,...,KATONG,"LIM PEI JUAN, TRACY",1187084260000,Kamsani Bin Afandi,1571136111000,180718-13187,61.342395,158.483763,221.877268,"POLYGON ((35940.580 32613.595, 35938.195 32611..."


In [ ]:
gpr.to_file(path+'/SG_GPR_sorted.geojson', driver='GeoJSON')

Now the GPR data has got sorted out. 